In [33]:
PROMPT = """
<|im_start|>system
As a geological expert, your task is to scrutinize the provided dataset for predefined geological relationships. Only the relationships listed below are acceptable:

    "has lithology of"
    "has sedimentary structure"
    "lithology has type of"
    "has grains of"
    "has color of"
    "has bedform of"
    "has structure of"
    
Disregard any relationships not explicitly listed, including those related to location or time, and avoid attributing new meanings to the specified relationships. Apply a rigorous standard in your assessment, promptly categorizing any divergent or ambiguous relationship as invalid.

Note that the head and tail may need to be swapped when you rewrite the relationship.

Please format your output as follows:

{{
  "reasoning": "Detail your reasoning behind the relationship's validity. State whether or not the relationship is valid at the end of your reasoning.",
  "is valid": "Indicate the relationship's alignment with the specified list",
  "head": "Specify the subject of the relationship",
  "tail": "Specify the object of the relationship",
  "relationship": "Cite the specific valid relationship from the list."
}}

<|im_end|>
<|im_start|>user
{{
  "head": "massive, miliolid, pelleted, shell-fragment wackestone and mudstone, grainstones",
  "tail": "West Nueces Formation",
  "relationship": "constitutes"
}}
<|im_end|>
<|im_start|>assistant
{{
  "reasoning": "Although 'constitutes' does not directly match 'has lithology of', the concept is similar. It implies that the wackestone, mudstone, and grainstones are components of the West Nueces Formation, aligning with the concept that the formation has a lithology characterized by these elements. Thus, the relationship is considered valid.",
  "is valid": true,
  "head": "West Nueces Formation",
  "tail": "massive, miliolid, pelleted, shell-fragment wackestone and mudstone, grainstones",
  "relationship": "has lithology of"
}}
<|im_end|>
<|im_start|>user
{{
  "head": "Glen Rose Formation", 
  "tail": "Montana", 
  "relationship": "is located in"
}}
<|im_end|>
<|im_start|>assistant
{{
  "reasoning": "The relationship 'is located in' describes a spatial or geographical association rather than a geological relationship pertaining to lithology, sedimentary structures, stratigraphic units, or other specified geological characteristics. Because this relationship found in the predefined list, the relationship is invalid.",
  "is valid": false
}}
<|im_end|>
<|im_start|>user
{{
  "head": "Cross Member", 
  "tail": "Comfort and Spring Garden Members of Castle Hayne Formation", 
  "relationship": "cross correlates with"
}}
<|im_end|>
<|im_start|>assistant
{{
  "reasoning": "The relationship 'cross correlates with' does not directly align with any of the specified valid relationships such as 'has lithology of', 'has sedimentary structure', 'stratigraphic unit (formation) has lithology', 'lithology has type of', 'has grains of', 'has color of', 'has bedform of', or 'has structure of'. The term 'cross correlates with' refers to a stratigraphic correlation between different members or parts of a formation, which is more related to the spatial and temporal relationship between geological units rather than their physical or compositional characteristics.",
  "is valid": false
}}
<|im_end|>
<|im_start|>user
{{
  "head": "biomicrite (limestone)", 
  "tail": "Cross Member", 
  "relationship": "is a component of"
}}
<|im_end|>
<|im_start|>assistant
{{
  "reasoning": "The phrase 'is a component of' suggests that the biomicrite (limestone) is part of the Cross Member, implying a composition relationship similar to 'has lithology of'. In geological terms, stating that a certain lithology (in this case, biomicrite, which is a type of limestone) is a component of a stratigraphic unit (Cross Member) aligns with the concept that the stratigraphic unit 'has lithology of' that limestone. Therefore, this relationship can be adapted to fit the 'stratigraphic unit (formation) has lithology' category, as it conveys that the Cross Member consists, at least in part, of biomicrite (limestone).",
  "is valid": true,
  "head": "Cross Member",
  "tail": "biomicrite (limestone)"
}}
<|im_end|>
<|im_start|>user
{input}
<|im_end|>
<|im_start|>assistant
"""

GRAMMAR = r"""
root   ::= object

object ::= 
  "{ \"reasoning\": " string ", \"is valid\": " ("true" | "false") ", \"head\": " string ", \"tail\": " string ", \"relationship\": " relationship " }"

relationship ::= "\"" ( "has lithology of" 
                       | "has sedimentary structure" 
                       | "stratigraphic unit has lithology" 
                       | "lithology has type of" 
                       | "has grains of" 
                       | "has color of" 
                       | "has bedform of" 
                       | "has structure of" ) "\""

invalidObject ::= "{ \"reasoning\":" string ", \"is valid\": false }"

string ::=
  "\"" (
    [^"\\] | "\\" (["\\/bfnrt] | "u" [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F])
  )* "\""
"""

In [34]:
"""
root ::= validObject | invalidObject

validObject ::= "{" ws "reasoning" ":" string "," ws "is valid" ":" "true" "," ws "head" ":" string "," ws "tail" ":" string "," ws "relationship" ":" relationship ws "}"

invalidObject ::= "{" ws "reasoning" ":" string "," ws "is valid" ":" "false" ws "}"

relationship ::= "\"" ( "has lithology of" 
                       | "has sedimentary structure" 
                       | "stratigraphic unit (formation) has lithology" 
                       | "lithology has type of" 
                       | "has grains of" 
                       | "has color of" 
                       | "has bedform of" 
                       | "has structure of" ) "\""

string ::=  "\"" (
        [^"\\] |
        "\\" (["\\/bfnrt] | "u" [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F])
      )* "\"" space

ws ::= " "?

space ::= " "?
string ::=  "\"" (
        [^"\\] |
        "\\" (["\\/bfnrt] | "u" [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F])
      )* "\"" space
boolean ::= ("true" | "false") space
root ::= "{" space "\"reasoning\"" space ":" space string "," space "\"is valid\"" space ":" space boolean "}" space
"""

'\nroot ::= validObject | invalidObject\n\nvalidObject ::= "{" ws "reasoning" ":" string "," ws "is valid" ":" "true" "," ws "head" ":" string "," ws "tail" ":" string "," ws "relationship" ":" relationship ws "}"\n\ninvalidObject ::= "{" ws "reasoning" ":" string "," ws "is valid" ":" "false" ws "}"\n\nrelationship ::= """ ( "has lithology of" \n                       | "has sedimentary structure" \n                       | "stratigraphic unit (formation) has lithology" \n                       | "lithology has type of" \n                       | "has grains of" \n                       | "has color of" \n                       | "has bedform of" \n                       | "has structure of" ) """\n\nstring ::=  """ (\n        [^"\\] |\n        "\\" (["\\/bfnrt] | "u" [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F])\n      )* """ space\n\nws ::= " "?\n\nspace ::= " "?\nstring ::=  """ (\n        [^"\\] |\n        "\\" (["\\/bfnrt] | "u" [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F])

In [35]:
import pandas as pd
import requests
import json
import ast
from IPython.display import display, HTML 

In [36]:
df = pd.read_csv('data/results_llm_240208.csv')

In [37]:
df.head()

,formation_name,paper_id,paragraph,triplet
0,Raysor Formation,55b735f3e13823bd29ba8028,"Cross Member (Santee Limestone) 1. Eocene, mid...","[{'head': 'Cross Member', 'tail': 'Santee Lime..."
1,Raysor Formation,557c3ad1e138239225f86760,The most frequently applied names for Pliocene...,"[{'head': 'Yorktown Formation', 'tail': 'Plioc..."
2,Raysor Formation,55b736e4e13823bd29ba802f,FIGURE 34.-Known distribution of Pliocene loca...,NaN
3,Raysor Formation,58cb666bcf58f10af549ae27,"Such an age range for the phosphate-beds ""faun...","[{'head': 'age range of phosphate-beds fauna',..."
4,Raysor Formation,55b736e4e13823bd29ba802f,Ward indicates that such shells were the sourc...,"[{'head': 'shells', 'tail': 'matrix', 'relatio..."


In [38]:
# Parse the string to a Python object
data = ast.literal_eval(df.iloc[0,3])

# Convert the Python object to a JSON string
print(json.dumps(data))

[{"head": "Cross Member", "tail": "Santee Limestone", "relationship": "is a type of"}, {"head": "Cross Member", "tail": "Eocene, middle (Claibornian)", "relationship": "belongs to"}, {"head": "Cross Member", "tail": "South Carolina", "relationship": "is located in"}, {"head": "Ward, L. W., and others, 1979, Stratigraphic revision of Eocene, Oligocene and lower Miocene formations of South Carolina", "tail": "Cross Member", "relationship": "provides information about"}, {"head": "biomicrite (limestone)", "tail": "Cross Member", "relationship": "is a component of"}, {"head": "grayish-yellow", "tail": "biomicrite (limestone)", "relationship": "describes"}, {"head": "1 m, range 1-41 m", "tail": "biomicrite (limestone)", "relationship": "describes"}, {"head": "bryozoan-brachiopod-bivalve biomicrite, deeply burrowed", "tail": "Cross Member", "relationship": "is a component of"}, {"head": "Moultrie Member (Santee Limestone)", "tail": "Cross Member", "relationship": "unconformably underlies"}, 

In [40]:
i = 0
results_df = pd.DataFrame(columns=["head", "tail", "relationship"])
for index, row in df.iterrows():
    try:
        relation_list = ast.literal_eval(df.iloc[0,3])
    except:
        print("Error: Could not parse input.")
        continue
    
    for relation in relation_list:
        try:
            data = {
                "temperature": 0.0,
                "prompt": PROMPT.format(input=json.dumps(relation)),
                "grammar": GRAMMAR,
            }
            
            response = requests.post("http://127.0.0.1:8000/completion", json=data)
            # print(response.json()["content"])
            response_json = json.loads(response.json()["content"])
            # print(relation)
            # print(response_json)
            if not response_json["is valid"]:
                continue
            
            results_df.loc[len(results_df.index)] = (
                    response_json["head"],
                    response_json["tail"],
                    response_json["relationship"],
                )
            
            if i % 5 == 0:
                print(f"Processed {i} relations. ")
                results_df.to_csv("output.csv", index=False)
                
            i += 1
            
        except KeyError: 
            print("Error: LLM output error. ")
            continue

Processed 0 relations. 
Processed 5 relations. 
Processed 10 relations. 


KeyboardInterrupt: 